<a href="https://colab.research.google.com/github/aliakbarbadri/persian-poetry-creator/blob/master/char-RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

# Load the data

In [0]:
# ! wget --no-check-certificate https://github.com/aliakbarbadri/persian-poetry-creator/blob/master/Shahnameh.zip?raw=true -O /tmp/Shahnameh.zip
# ! unzip /tmp/Shahnameh.zip 

In [0]:
# import os
# corpus_url = "Shahnameh/shahname"
# corpus = ""
# for filename in os.listdir(corpus_url):
#    with open(os.path.join(corpus_url, filename), 'r') as f:
#       text = f.read()
#       corpus += text
#       corpus += "\n"

In [0]:
# file_corpus = open("shahname.txt", "a")
# file_corpus.write(corpus)
# file_corpus.close()

In [0]:
url = "https://raw.githubusercontent.com/aliakbarbadri/persian-poetry-creator/master/shahname.txt"
filepath = keras.utils.get_file("shahname.txt", url) 
with open(filepath) as f:
  corpus = f.read()

In [6]:
len(corpus)

2555369

In [0]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True) 
tokenizer.fit_on_texts([corpus])

In [8]:
tokenizer.texts_to_sequences(["سلام"])

[[14, 25, 2, 10]]

In [9]:
tokenizer.sequences_to_texts([[14, 25, 2, 10]])

['س ل ا م']

In [10]:
"".join(sorted(set(corpus)))

'\t\n ()«»،؟ءآأؤئابتثجحخدذرزسشصضطظعغفقلمنهؤپچژکگی\u200c'

In [11]:
max_id = len(tokenizer.word_index)
dataset_size = len(corpus)
max_id,dataset_size

(48, 2555369)

In [0]:
[encoded] = np.array(tokenizer.texts_to_sequences([corpus])) - 1

In [0]:
train_size = dataset_size * 90 // 100

In [0]:
n_steps = 100
window_length = n_steps + 1 # target = input shifted 1 character ahead 
batch_size = 32

# Stateless RNN

In [0]:
# dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
# dataset = dataset.window(window_length, shift=1, drop_remainder=True)
# dataset = dataset.flat_map(lambda window: window.batch(window_length))
# dataset = dataset.shuffle(10000).batch(batch_size)
# dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
# dataset = dataset.map(lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
# dataset = dataset.prefetch(1)
# for X_batch, Y_batch in dataset.take(1):
#     print(X_batch.shape, Y_batch.shape)

In [0]:
# model = keras.models.Sequential([
#     tf.compat.v1.keras.layers.CuDNNGRU(128, return_sequences=True, 
#                                        input_shape=[None, max_id],
#                                        recurrent_regularizer=keras.regularizers.l2(1e-4)),
#     tf.compat.v1.keras.layers.CuDNNGRU(128, return_sequences=True,
#                                        recurrent_regularizer=keras.regularizers.l2(1e-4)),
#     keras.layers.TimeDistributed(keras.layers.Dense(max_id,
#                                                     activation="softmax"))
# ])
# model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")

In [0]:
# history = model.fit(dataset, steps_per_epoch=train_size // batch_size,epochs=10)

# Stateful RNN

In [0]:
tf.random.set_seed(42)

In [0]:
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True) 
dataset = dataset.flat_map(lambda window: window.batch(window_length)) 
dataset = dataset.batch(1)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:])) 
dataset = dataset.map(lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch)) 
dataset = dataset.prefetch(1)

In [0]:
batch_size = 32
encoded_parts = np.array_split(encoded[:train_size], batch_size)
datasets = []
for encoded_part in encoded_parts:
    dataset = tf.data.Dataset.from_tensor_slices(encoded_part)
    dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_length))
    datasets.append(dataset)
dataset = tf.data.Dataset.zip(tuple(datasets)).map(lambda *windows: tf.stack(windows))
dataset = dataset.repeat().map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

In [0]:
model = keras.models.Sequential([
  keras.layers.GRU(128, return_sequences=True, stateful=True,recurrent_regularizer=keras.regularizers.l2(1e-2),
                    batch_input_shape=[batch_size, None, max_id]), 
  keras.layers.GRU(128, return_sequences=True, stateful=True,recurrent_regularizer=keras.regularizers.l2(1e-2)),                   
  keras.layers.TimeDistributed(keras.layers.Dense(max_id,activation="softmax"))
])

In [0]:
class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

In [0]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
steps_per_epoch = train_size // batch_size // n_steps

In [21]:
history = model.fit(dataset, steps_per_epoch=steps_per_epoch, epochs=50, callbacks=[ResetStatesCallback()])

Epoch 1/50
718/718 [==============================] - 35s 49ms/step - loss: 2.5722
Epoch 2/50
718/718 [==============================] - 35s 49ms/step - loss: 2.1264
Epoch 3/50
718/718 [==============================] - 35s 49ms/step - loss: 1.9908
Epoch 4/50
718/718 [==============================] - 35s 49ms/step - loss: 1.9233
Epoch 5/50
718/718 [==============================] - 35s 49ms/step - loss: 1.8767
Epoch 6/50
718/718 [==============================] - 35s 49ms/step - loss: 1.8403
Epoch 7/50
718/718 [==============================] - 35s 49ms/step - loss: 1.8101
Epoch 8/50
718/718 [==============================] - 35s 49ms/step - loss: 1.7858
Epoch 9/50
718/718 [==============================] - 35s 49ms/step - loss: 1.7648
Epoch 10/50
718/718 [==============================] - 35s 49ms/step - loss: 1.7463
Epoch 11/50
718/718 [==============================] - 35s 49ms/step - loss: 1.7299
Epoch 12/50
718/718 [==============================] - 35s 49ms/step - loss: 1.7150
E

In [0]:
stateless_model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,activation="softmax"))
])

In [0]:
stateless_model.build(tf.TensorShape([None, None, max_id]))

In [0]:
stateless_model.set_weights(model.get_weights())
model = stateless_model

In [0]:
model.save("char_rnn_gru.h5")

# Use model

In [0]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

In [26]:
import warnings
warnings.filterwarnings('ignore')

X_new = preprocess(["که"])
Y_pred = model.predict_classes(X_new)
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1] # 1st sentence, last char

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


' '

In [27]:
tf.random.set_seed(42)
tf.random.categorical([[np.log(0.5), np.log(0.4), np.log(0.1)]], num_samples=40).numpy()

array([[0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0,
        2, 0, 0, 1, 1, 1, 0, 0, 1, 2, 0, 0, 1, 1, 0, 0, 0, 0]])

In [0]:
def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model.predict(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [29]:
tf.random.set_seed(42)
next_char("که ایران چو با", temperature=1)

'\n'

In [0]:
def complete_text(text, n_chars=100, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [32]:
tf.random.set_seed(42)
print(complete_text("که ایران چو باغی ست خرم بهار", temperature=1))

که ایران چو باغی ست خرم بهار
پس از دیدم و گنجی بدیدش‌ه	همان غم جلست و بنگره
دگرگون چون که دارا گهر جانت	جهابد رسیدار و زیربفشاند
